In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [5]:
frame_folders = [f for f in Path('./frames').glob("*") if f.is_dir()]
output_folder = Path('./output')
final_csv = Path('./final_csv')
final_csv.mkdir(exist_ok=True)

In [6]:
def merge_file(cut_imgs,df1,df2):
    ids = [img.name for img in cut_imgs]
    df = pd.DataFrame(data={'frame':ids})
    df = pd.merge(df,df1,on='frame',how='left')
    df = pd.merge(df,df2,on='frame',how='left')
    return df
past_value=np.nan

def fill_nan(value):
    global past_value
    if pd.isna(value):
        return past_value
    else:
        past_value = value
        return value

In [7]:
for frame_folder in tqdm(frame_folders):
    vid = frame_folder.name
    frame_imgs = sorted(list(frame_folder.glob("*.png")))
    anno_file = output_folder/f'{vid}_anno.pkl'
    view_file = output_folder/f'{vid}_result.csv'
    select_file = output_folder/f'{vid}_selection.csv'
    df_view = pd.read_csv(view_file)
    df_select = pd.read_csv(select_file)
    df = merge_file(frame_imgs,df_view,df_select)
    past_value=np.nan
    df['viewport'] = df.viewport.apply(fill_nan)
    past_value=np.nan
    df['selection'] = df.selection.apply(fill_nan)
    df.to_csv(final_csv/f'{vid}.csv',index=False)

  0%|          | 0/23 [00:00<?, ?it/s]